In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

In [2]:
doc = pd.read_csv("COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/08-05-2022.csv", encoding='utf-8-sig') #데이터 불러오기

In [3]:
doc.head() #처음 5개 데이터 확인

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-08-06 04:20:56,33.93911,67.709953,187037,7752,NaN,NaN,Afghanistan,480.464862,4.144634
1,NaN,NaN,NaN,Albania,2022-08-06 04:20:56,41.15330,20.168300,316145,3557,NaN,NaN,Albania,10985.648759,1.125117
2,NaN,NaN,NaN,Algeria,2022-08-06 04:20:56,28.03390,1.659600,268033,6878,NaN,NaN,Algeria,611.235176,2.566102
3,NaN,NaN,NaN,Andorra,2022-08-06 04:20:56,42.50630,1.521800,45793,154,NaN,NaN,Andorra,59267.456157,0.336296
4,NaN,NaN,NaN,Angola,2022-08-06 04:20:56,-11.20270,17.873900,102636,1917,NaN,NaN,Angola,312.283707,1.867766


In [4]:
doc.info() #데이터 타입과 실제 데이터가 있는 사이즈 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4012 entries, 0 to 4011
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FIPS                 3268 non-null   float64
 1   Admin2               3272 non-null   object 
 2   Province_State       3835 non-null   object 
 3   Country_Region       4012 non-null   object 
 4   Last_Update          4012 non-null   object 
 5   Lat                  3921 non-null   float64
 6   Long_                3921 non-null   float64
 7   Confirmed            4012 non-null   int64  
 8   Deaths               4012 non-null   int64  
 9   Recovered            0 non-null      float64
 10  Active               0 non-null      float64
 11  Combined_Key         4012 non-null   object 
 12  Incident_Rate        3918 non-null   float64
 13  Case_Fatality_Ratio  3969 non-null   float64
dtypes: float64(7), int64(2), object(5)
memory usage: 438.9+ KB


# mini assignments

## 1. 가장 최근 dataset적용 코로나 확진자수 시각화

In [5]:
with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file: #변경 할 국가명이 있는 json파일 불러오기
    json_data = json.load(json_file)
      
def country_name_convert(row): #Country_Region컬럼값을 확인해서 Country_Region이 다를 경우 지정한 국가명으로 변경
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']


def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') # csv 파일 불러오기
    try:
        doc = doc[['Country_Region', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성
    except:
        doc = doc[['Country/Region', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성 및 컬럼명 변경
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed']) # 특정 컬럼에 없는 데이터 삭제
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1) # 'Country_Region'의 국가명을 여러 파일에 일관되게 변경
    doc = doc.astype({'Confirmed': 'int64'}) # Confirmed 컬럼 데이터 타입 int64로 변경
    doc = doc.groupby('Country_Region').sum() # Country_Region컬럼 기준으로 데이터 합치기

    date_column = filename.split(".")[0].replace('-', '/') 
    date_column = date_column.split('/')[2]+'/'+date_column.split('/')[0]+'/'+date_column.split('/')[1]
    doc.columns = [date_column] #날짜 변환 및 컬럼명 변경
    return doc

In [6]:
def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv': #확장자가 csv인 파일명만 불러오기
            csv_list.append(file) #csv_list에 추가
    csv_list.sort() #오름차순으로 정렬
    
    for file in csv_list:
        doc = create_dateframe(file) #데이터프레임 전처리
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True) #데이터 프레임 합치기

    final_doc = final_doc.fillna(0) #결측값 0으로 대체
    return final_doc

In [7]:
def create_flag_link(row):
    flag_link = 'https://flagcdn.com/48x36/' + row + '.png'
    return flag_link #국기 사진 링크 생성


PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
df_confirmed = generate_dateframe_by_path(PATH)
df_confirmed = df_confirmed.astype('int64') #df_confirmed 컬럼 데이터 타입 int64로 변경

country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')#국가명과 iso2 매칭 테이블 불러오기
country_info = country_info[['iso2', 'Country_Region']]
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')

doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region') #데이터프래임 합치기
doc_final_country = doc_final_country.dropna(subset=['iso2'])
doc_final_country['iso2'] = doc_final_country['iso2'].str.lower().apply(create_flag_link) #iso2 컬럼 추가

cols = doc_final_country.columns.tolist() #컬럼명 리스트로 변환
cols.sort() #오름차순 정렬

cols.remove('iso2') #컬럼 위치 변경
cols.insert(0, 'iso2')

cols.remove('Country_Region') #컬럼 위치 변경
cols.insert(0, 'Country_Region')

doc_final_country = doc_final_country[cols]
cols[1] = 'Country_Flag'
doc_final_country.columns = cols #컬럼명 변경

doc_final_country.to_csv("graph/final_covid_data_for_graph_Confirmed.csv") #최종 파일 csv로 저장

## 2. 가장 최근 dataset적용 코로나 사망자 시각화

In [8]:
with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file: #json 파일 불러오기
    json_data = json.load(json_file)
      
def country_name_convert(row): #Country_Region컬럼값을 확인해서 Country_Region이 다를 경우 지정한 국가명으로 변경
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']


def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') #csv 파일 불러오기
    try:
        doc = doc[['Country_Region', 'Deaths']] # 특정 컬럼 선택해서 데이터프레임 생성
    except:
        doc = doc[['Country/Region', 'Deaths']] # 특정 컬럼 선택해서 데이터프레임 생성 및 컬럼명 변경
        doc.columns = ['Country_Region', 'Deaths']
    doc = doc.dropna(subset=['Deaths']) # 특정 컬럼에 없는 데이터 삭제
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1) # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경
    doc = doc.astype({'Deaths': 'int64'}) #Deaths 컬럼 데이터 타입 int64로 변경
    doc = doc.groupby('Country_Region').sum() #Country_Region 컬럼 기준으로 중복된 데이터를 합치기

    date_column = filename.split(".")[0].replace('-', '/')
    date_column = date_column.split('/')[2]+'/'+date_column.split('/')[0]+'/'+date_column.split('/')[1]
    doc.columns = [date_column] #날짜 변환 및 컬럼명 변경
    return doc

In [9]:
def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv': #확장자가 csv인 파일명만 불러오기
            csv_list.append(file) #csv_list에 추가
    csv_list.sort() #오름차순으로 정렬
    
    for file in csv_list:
        doc = create_dateframe(file) #데이터프레임 전처리
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0) #결측값 0으로 대체
    return final_doc

In [10]:
def create_flag_link(row):
    flag_link = 'https://flagcdn.com/48x36/' + row + '.png'
    return flag_link #국기 사진 링크 생성


PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
df_Deaths = generate_dateframe_by_path(PATH)
df_Deaths = df_Deaths.astype('int64') #df_Deaths 컬럼 데이터 타입 int64로 변경

country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='') #국가명과 iso2 매칭 테이블 불러오기
country_info = country_info[['iso2', 'Country_Region']]
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')

doc_final_country = pd.merge(df_Deaths, country_info, how='left', on='Country_Region') #데이터프래임 합치기
doc_final_country = doc_final_country.dropna(subset=['iso2'])
doc_final_country['iso2'] = doc_final_country['iso2'].str.lower().apply(create_flag_link) #iso2 컬럼 추가

cols = doc_final_country.columns.tolist() #컬럼명 리스트로 변환
cols.sort() #오름차순 정렬

cols.remove('iso2') #컬럼 위치 변경
cols.insert(0, 'iso2')

cols.remove('Country_Region') #컬럼 위치 변경
cols.insert(0, 'Country_Region')

doc_final_country = doc_final_country[cols]
cols[1] = 'Country_Flag'
doc_final_country.columns = cols

doc_final_country.to_csv("graph/final_covid_data_for_graph_Deaths.csv") #최종 파일 csv로 저장

## 3.가장 최근 dataset적용 코로나 회복자 시각화

In [11]:
with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file: #json 파일 불러오기
    json_data = json.load(json_file)
      
def country_name_convert(row): #Country_Region컬럼값을 확인해서 Country_Region이 다를 경우 지정한 국가명으로 변경
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']


def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') #csv 파일 불러오기
    try:
        doc = doc[['Country_Region', 'Recovered']] # 특정 컬럼 선택해서 데이터프레임 생성
    except:
        doc = doc[['Country/Region', 'Recovered']] # 특정 컬럼 선택해서 데이터프레임 생성 및 컬럼명 변경
        doc.columns = ['Country_Region', 'Recovered']
    doc = doc.dropna(subset=['Recovered']) # 특정 컬럼에 없는 데이터 삭제
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)# 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경
    doc = doc.astype({'Recovered': 'int64'}) #Recovered 컬럼 데이터 타입 int64로 변경
    doc = doc.groupby('Country_Region').sum() #Country_Region 컬럼 기준으로 중복된 데이터를 합치기

    date_column = filename.split(".")[0].replace('-', '/')
    date_column = date_column.split('/')[2]+'/'+date_column.split('/')[0]+'/'+date_column.split('/')[1]
    doc.columns = [date_column] #날짜 변환 및 컬럼명 변경
    return doc

In [12]:
def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv': #확장자가 csv인 파일명만 불러오기
            csv_list.append(file) #csv_list에 추가
    csv_list.sort() #오름차순으로 정렬
    
    for file in csv_list:
        doc = create_dateframe(file) #데이터프레임 전처리
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True) #데이터 프레임 합치기

    final_doc = final_doc.fillna(0) #결측값 0으로 대체
    return final_doc

In [13]:
def create_flag_link(row):
    flag_link = 'https://flagcdn.com/48x36/' + row + '.png'
    return flag_link #국기 사진 링크 생성


PATH = 'csse_covid_19_data/csse_covid_19_daily_reports/'
df_Recovered = generate_dateframe_by_path(PATH)
df_Recovered = df_Recovered.astype('int64') #df_Recovered 컬럼 데이터 타입 int64로 변경

country_info = pd.read_csv("csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='') #국가명과 iso2 매칭 테이블 불러오기
country_info = country_info[['iso2', 'Country_Region']]
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')

doc_final_country = pd.merge(df_Recovered, country_info, how='left', on='Country_Region') #데이터프래임 합치기
doc_final_country = doc_final_country.dropna(subset=['iso2'])
doc_final_country['iso2'] = doc_final_country['iso2'].str.lower().apply(create_flag_link) #iso2 컬럼 추가

cols = doc_final_country.columns.tolist() #컬럼명 리스트로 변환
cols.sort() #오름차순 정렬

cols.remove('iso2') #컬럼 위치 변경
cols.insert(0, 'iso2')

cols.remove('Country_Region') #컬럼 위치 변경
cols.insert(0, 'Country_Region')

doc_final_country = doc_final_country[cols]
cols[1] = 'Country_Flag'
doc_final_country.columns = cols

doc_final_country.to_csv("graph/final_covid_data_for_graph_Recovered.csv") #최종 파일 csv로 저장

# mini competition

## 1. 가장 최근 dataset 적용 미국내 주별 코로나 확진자수 시각화

In [14]:
with open('COVID-19-master/csse_covid_19_data/US_convert.json', 'r', encoding='utf-8-sig') as json_file: #변경 할 국가명이 있는 json파일 불러오기
    json_data=json.load(json_file)    
    
def country_name_convert(row): #Province_State컬럼값을 확인해서 Province_State이 다를 경우 지정한 국가명으로 변경
    if row['Province_State'] in json_data:
        return json_data[row['Province_State']]
    return row['Province_State']

def create_dateframe(filename):   
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') # csv 파일 불러오기
    try:
        doc = doc[['Province_State', 'Country_Region', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성
    except:
        doc = doc[['Province/State', 'Country/Region', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성 및 컬럼명 변경
        doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed']) # 특정 컬럼에 없는 데이터 삭제
    doc['State_Code'] = doc.apply(country_name_convert, axis=1)  
    doc = doc.astype({'Confirmed' :'int64'}) # Confirmed 컬럼 데이터 타입 int64로 변경
    doc = doc.groupby('Province_State').sum() # Province_State컬럼 기준으로 데이터 합치기
    
    date_column = filename.split(".")[0].replace('-', '/')
    date_column = date_column.split('/')[2]+'/'+date_column.split('/')[0]+'/'+date_column.split('/')[1]
    doc.columns = [date_column] #날짜 변환 및 컬럼명 변경
    return doc

In [15]:
def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv': #확장자가 csv인 파일명만 불러오기
            csv_list.append(file) #csv_list에 추가
    csv_list.sort() #오름차순으로 정렬
    
    for file in csv_list:
        doc = create_dateframe(file) #데이터프레임 전처리
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True) #데이터 프레임 합치기

    final_doc = final_doc.fillna(0) #결측값 0으로 대체
    return final_doc

In [16]:
def create_flag_link(row):
    flag_link = 'https://flagcdn.com/48x36/us-' + row + '.png'
    return flag_link #미국주 사진 링크 생성

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports_us/'
df_confirmed = generate_dateframe_by_path(PATH)
df_confirmed = df_confirmed.astype('int64')#df_confirmed 컬럼 데이터 타입 int64로 변경

Province_info = pd.read_csv("COVID-19-master\csse_covid_19_data/US_ABBREV_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='') #국가명과 US_abb 매칭 테이블 불러오기
Province_info = Province_info[['US_abb', 'Province_State']]
Province_info = Province_info.drop_duplicates(subset='Province_State', keep='last')

doc_final_Province = pd.merge(df_confirmed, Province_info, how='left', on='Province_State') #데이터프래임 합치기
doc_final_Province = doc_final_Province.dropna(subset=['US_abb'])
doc_final_Province['US_abb'] = doc_final_Province['US_abb'].str.lower().apply(create_flag_link) #US_abb 컬럼 추가

cols = doc_final_Province.columns.tolist() #컬럼명 리스트로 변환
cols.sort() #오름차순 정렬

cols.remove('US_abb') #컬럼 위치 변경
cols.insert(0, 'US_abb')

cols.remove('Province_State') #컬럼 위치 변경
cols.insert(0, 'Province_State')

doc_final_Province = doc_final_Province[cols]
cols[1] = 'Province_State_Flag'
doc_final_Province.columns = cols

doc_final_Province.to_csv("graph/final_covid_data_for_graph_US.csv") #최종 파일 csv로 저장

## 2. Dummy Bonus Points 중국내 도시별 코로나 확진자수 시각화. 

In [17]:
with open('COVID-19-master/csse_covid_19_data/CN_convert.json', 'r', encoding='utf-8-sig') as json_file:  #변경 할 국가명이 있는 json파일 불러오기
    json_data = json.load(json_file) 
      
def country_name_convert(row): #Province_State컬럼값을 확인해서 Province_State이 다를 경우 지정한 국가명으로 변경
    if row['Province_State'] in json_data:
        return json_data[row['Province_State']]
    return row['Province_State']

def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig') # csv 파일 불러오기
    try:
        doc = doc[['Country_Region', 'Province_State', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성
    except:
        doc = doc[['Country/Region', 'Province/State', 'Confirmed']] # 특정 컬럼만 선택해서 데이터프레임 생성 및 컬럼명 변경
        doc.columns = ['Country_Region', 'Province_State', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed']) # 특정 컬럼에 없는 데이터 삭제
    doc['State_code'] = doc.apply(country_name_convert, axis=1) 
    doc = doc.astype({'Confirmed': 'int64'}) # Confirmed 컬럼 데이터 타입 int64로 변경
    doc = doc.groupby('Province_State').sum() # Province_State컬럼 기준으로 데이터 합치기

    date_column = filename.split(".")[0].replace('-', '/')
    date_column = date_column.split('/')[2]+'/'+date_column.split('/')[0]+'/'+date_column.split('/')[1]
    doc.columns = [date_column] #날짜 변환 및 컬럼명 변경
    return doc

In [18]:
def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv': #확장자가 csv인 파일명만 불러오기
            csv_list.append(file) #csv_list에 추가
    csv_list.sort() #오름차순으로 정렬
    
    for file in csv_list:
        doc = create_dateframe(file) #데이터프레임 전처리
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True) #데이터 프레임 합치기

    final_doc = final_doc.fillna(0) #결측값 0으로 대체
    return final_doc

In [19]:
def create_flag_link(row):
    flag_link = 'https://flagcdn.com/48x36/'+row +'.png'
    return flag_link #중국 사진 링크 생성


PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
df_confirmed = generate_dateframe_by_path(PATH)
df_confirmed = df_confirmed.astype('int64') #df_confirmed 컬럼 데이터 타입 int64로 변경

Province_info = pd.read_csv("COVID-19-master/csse_covid_19_data/CN_CD_LookUP_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
Province_info = Province_info[['ch_cd', 'Province_State']]
Province_info = Province_info.drop_duplicates(subset='Province_State', keep='last')

doc_final_Province = pd.merge(df_confirmed, Province_info, how='left', on='Province_State')
doc_final_Province = doc_final_Province.dropna(subset=['ch_cd'])
doc_final_Province['ch_cd'] = doc_final_Province['ch_cd'].str.lower().apply(create_flag_link)

cols = doc_final_Province.columns.tolist()
cols.sort()

cols.remove('ch_cd')
cols.insert(0, 'ch_cd')

cols.remove('Province_State')
cols.insert(0, 'Province_State')

doc_final_Province = doc_final_Province[cols]
cols[1] = 'Country_Flag'
doc_final_Province.columns = cols

doc_final_Province.to_csv("graph/final_covid_data_for_graph_CN.csv") #최종 파일 csv로 저장

## 3. 서울시 코로나19 자치구별 확진자 발생동향

In [20]:
doc = pd.read_csv("covid19_seoul.csv", encoding='cp949') #데이터 불러오기

In [21]:
doc.head() #처음 5개 데이터 확인

,자치구 기준일,종로구 전체,종로구 추가,중구 전체,중구 추가,용산구 전체,용산구 추가,성동구 전체,성동구 추가,광진구 전체,...,서초구 전체,서추구 추가,강남구 전체,강남구 추가,송파구 전체,송파구 추가,강동구 전체,강동구 추가,기타 전체,기타 추가
0,2022.08.11.00,56798,315,51319,316,87935,592,122189,675,147602,...,167904,950,216569,1197,274833,1693,191265,1047,259571,1544
1,2022.08.10.00,56483,333,51003,292,87343,599,121514,668,146760,...,166954,1042,215372,1327,273140,1713,190218,1215,258027,1749
2,2022.08.09.00,56150,371,50711,309,86744,661,120846,857,145970,...,165912,985,214045,1558,271430,2014,189004,1339,256281,1717
3,2022.08.08.00,55781,96,50403,91,86083,143,119990,165,145004,...,164928,366,212487,483,269416,523,187666,445,254568,404
4,2022.08.07.00,55685,262,50312,244,85940,388,119825,428,144708,...,164562,775,212004,869,268893,1128,187221,886,254164,928


In [22]:
doc.info() #데이터 타입과 실제 데이터가 있는 사이즈 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 53 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   자치구 기준일  920 non-null    object
 1   종로구 전체   920 non-null    int64 
 2   종로구 추가   920 non-null    int64 
 3   중구 전체    920 non-null    int64 
 4   중구 추가    920 non-null    int64 
 5   용산구 전체   920 non-null    int64 
 6   용산구 추가   920 non-null    int64 
 7   성동구 전체   920 non-null    int64 
 8   성동구 추가   920 non-null    int64 
 9   광진구 전체   920 non-null    int64 
 10  광진구 추가   920 non-null    int64 
 11  동대문구 전체  920 non-null    int64 
 12  동대문구 추가  920 non-null    int64 
 13  중랑구 전체   920 non-null    int64 
 14  중랑구 추가   920 non-null    int64 
 15  성북구 전체   920 non-null    int64 
 16  성북구 추가   920 non-null    int64 
 17  강북구 전체   920 non-null    int64 
 18  강북구 추가   920 non-null    int64 
 19  도봉구 전체   920 non-null    int64 
 20  도봉구 추가   920 non-null    int64 
 21  노원구 전체   920 non-null    int64 
 22  노원

In [23]:
doc = doc.loc[::-1].reset_index(drop=True)#자기구 기준일 컬럼값 역순 정렬
doc = doc.set_index('자치구 기준일') #자치구 기준일컬럼 인덱스로 지정
doc=doc.T #행과 열 바꾸기
doc = doc.reset_index(drop=False) #자치구 기준일 인덱스가 아닌 컬럼으로 지정하기 위해 인덱스 생성
doc=doc.rename(columns={'index':'자치구 기준일'}) #컬럼명 변경
doc = doc[doc['자치구 기준일'].str.contains('전체')] #컬럼값에 '전체'가 들어간 행만 출력

In [24]:
doc.head() #변경된 데이터 확인

자치구 기준일,자치구 기준일,20.02.05.10,20.02.06.10,20.02.07.10,20.02.08.10,20.02.09.10,20.02.10.10,20.02.11.10,20.02.12.10,20.02.13.10,...,2022.08.02.00,2022.08.03.00,2022.08.04.00,2022.08.05.00,2022.08.06.00,2022.08.07.00,2022.08.08.00,2022.08.09.00,2022.08.10.00,2022.08.11.00
0,종로구 전체,3,3,0,0,0,0,0,0,0,...,54330,54635,54916,55195,55423,55685,55781,56150,56483,56798
2,중구 전체,0,0,0,0,0,0,0,0,0,...,49126,49413,49636,49851,50068,50312,50403,50711,51003,51319
4,용산구 전체,0,0,0,0,0,0,0,0,0,...,83701,84216,84670,85155,85552,85940,86083,86744,87343,87935
6,성동구 전체,0,0,0,0,0,0,0,0,0,...,117172,117776,118272,118830,119397,119825,119990,120846,121514,122189
8,광진구 전체,0,0,0,0,0,0,0,0,0,...,141150,142027,142687,143395,144106,144708,145004,145970,146760,147602


In [25]:
doc.to_csv("graph/covid19_seoul_graph.csv", encoding='cp949') #최종 파일 csv로 저장